In [79]:
import os
import os.path as osp
import sys
sys.path.append("..")

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

import core
import utils

In [80]:
os.environ["TORCH_HOME"] = "."

In [81]:
model = torch.hub.load('hub/deeplabv3', source="local", model='deeplabv3_resnet50', pretrained=True)

In [82]:
print(model)
print(model.aux_classifier)

DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Se

output['out'] contains the semantic masks, and output['aux'] contains the auxiliary loss values per-pixel. In inference mode, output['aux'] is not useful. So, output['out'] is of shape (N, 21, H, W)

In [83]:
model.aux_classifier[-1] = nn.Conv2d(256, 9, kernel_size=(1, 1), stride=(1, 1))
model.classifier[-1] = nn.Conv2d(256, 9, kernel_size=(1, 1), stride=(1, 1))
stretch = nn.Upsample(scale_factor=(4, 1), mode='bilinear', align_corners=False)
squeeze = nn.Conv2d(5, 3, kernel_size=3, stride=1, padding=1)

In [84]:
import torch
import torch.nn as nn

# 创建一个 64x512 的随机特征图
x = torch.randn(4, 5, 64, 512)  # (batch_size, channels, height, width)

x = stretch(x)
x = squeeze(x)

model.train()
x = model(x)
out, aux = x["out"], x["aux"]

out = F.upsample(out, size=(64, 512))
aux = F.upsample(aux, size=(64, 512))

y = torch.randint(0, 9, (4, 64, 512))
criterion = nn.CrossEntropyLoss()
loss1 = criterion(out, y)
loss2 = criterion(aux, y)